In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import plotly.graph_objs as go
import pandas as pd
from datetime import datetime

import yfinance as yf

# Read data
df = pd.read_csv('../DATA/chilean_stocks.csv', index_col='Date')
df = df.groupby(by=['Ticker', 'Name']).sum().reset_index()[['Ticker','Name']].set_index('Ticker')

available_tickers = []
for tic in df.index:
    available_tickers.append({'label':df.loc[tic]['Name'], 'value':tic})

# Release memory
del df

app = dash.Dash(
    __name__, meta_tags=[{"name": "viewport", "content": "width=device-width"}]
)

available_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Pct Change - Close', 'Cummulative Returns']

app.layout = html.Div([
    html.H1('Bolsa de Santiago - Principales Acciones - Dashboard',
            style={'textAlign': 'center'},
           ),
    html.Div([html.H3(html.Label('Seleccionar la Acción')),
            dcc.Dropdown(
                id='ticker-selected',
                options=available_tickers,
                value='LTM.SN',
                multi=True,
                placeholder="Acción",
                clearable=False,
            )
    ],style={'width': '48%', 'display': 'inline-block'},),
    html.Div([html.H3(html.Label('Elija Información a visualizar')),
            dcc.Dropdown(
                id='y-selected',
                options=[{'label': i, 'value': i} for i in available_columns],
                value='Close',
                placeholder="Información a visualizar",
                clearable=False,
            )
    ],style={'width': '48%', 'display': 'inline-block'},),
    html.Div([
        html.H3('Seleccione fecha de inicio y final:'),
        dcc.DatePickerRange(
            id='my_date_picker',
            min_date_allowed=datetime(2018, 1, 1),
            max_date_allowed=datetime.today(),
            start_date=datetime(2020, 1, 1),
            end_date=datetime.today()
        )
    ], style={'display':'inline-block'}),
    html.Div([
        html.Button(
            id='submit-button',
            n_clicks=0,
            children='Submit',
            style={'fontSize':24, 'marginLeft':'30px'}
        ),
    ], style={'display':'inline-block'}),
    # Graph 
    html.Hr(),
    html.Div([dcc.Graph(id='graph-stock',
                        figure={
                            'data': [
                                {'x': [0,0], 'y': [0,0]}
                            ]
                        }),                         
             ],),
    html.Div([dcc.Graph(id='table-stock',
                        figure={
                            'data': [
                                {'x': [0,0], 'y': [0,0]}
                            ]
                        }),                         
             ],),
    html.Hr(),
])

@app.callback(Output('graph-stock', 'figure'),
              [Input('submit-button', 'n_clicks')],
              [State('ticker-selected', 'value'),
               State('y-selected', 'value'),
               State('my_date_picker', 'start_date'),
               State('my_date_picker', 'end_date'),
              ]
             )


def update_graph(n_clicks, tickers, col, start_date, end_date):
    start = datetime.strptime(start_date[:10], '%Y-%m-%d')
    end = datetime.strptime(end_date[:10], '%Y-%m-%d')
    traces = []
    title = ''
    if type(tickers)==str: tickers = [tickers]
    #tickers.append('BOLSASTGO.SN')
    for accion in tickers:
        df_ticker = yf.Ticker(accion)
        # get company name

        try:
            longname = df_ticker.info['longName']
            industry = df_ticker.info['industry']
            payoutRatio = df_ticker.info['payoutRatio']
        except:
            longname = accion
            industry = 'N/A'
            payoutRatio = 0 
        # get historical market data
        df = df_ticker.history(start=start, end=end, interval='1d')
        df['Ticker'] = accion
        df['Name'] = longname
        df['Industry'] = industry
        df['PayoutRatio'] = payoutRatio
        df['Pct Change - Close'] = df.Close.pct_change()
        df['Cummulative Returns'] = ((1 + df['Pct Change - Close']).cumprod() - 1)    

        data = go.Scatter(x = df.index, y=df[col], mode='lines+markers', 
                          name=accion,
                          text=longname, 
                          hovertext=["x", "text", col],
                          hoverinfo=["y"],
                          xaxis='x2',
                          yaxis='y2',
                          marker=dict(symbol = 'pentagon-dot',
                                      opacity = 0.8,
                                      line   = dict(width=2),
                                     ),
                         )
        traces.append(data)
        title = title + accion+'/'
    layout = go.Layout(title = title+' - '+'<b>'+col+'</b>'+'<br>'+'Desde : '+start_date[:10] \
                       + ' Hasta : '+end_date[:10],
                       title_x=0.5,
                       xaxis = dict(title='Fecha'),
                       yaxis = dict(title=col),
                       hovermode="x unified",
                       hoverlabel=dict(bgcolor="white", 
                                       font_size=10, 
                                       font_family="Rockwell"
                                      ),
                       template='presentation',
                       height=600,
                       autosize=True,  
                      )
    fig = go.Figure(data=traces,layout=layout)
    fig.update_xaxes(rangeslider_visible=True,
                     rangeselector=dict(buttons=list([
                         dict(count=1, label="1m", step="month", stepmode="backward"),
                         dict(count=6, label="6m", step="month", stepmode="backward"),
                         dict(count=1, label="YTD", step="year", stepmode="todate"),
                         dict(count=1, label="1y", step="year", stepmode="backward"),
                         dict(count=3, label="3y", step="year", stepmode="backward"),
                         dict(count=5, label="5y", step="year", stepmode="backward"),
                         dict(step="all")])))
    return fig

@app.callback(Output('table-stock', 'figure'),
              [Input('submit-button', 'n_clicks')],
              [State('ticker-selected', 'value'),
               State('y-selected', 'value'),
               State('my_date_picker', 'start_date'),
               State('my_date_picker', 'end_date'),
              ]
             )

def update_table(n_clicks, tickers, col, start_date, end_date):
    start = datetime.strptime(start_date[:10], '%Y-%m-%d')
    end = datetime.strptime(end_date[:10], '%Y-%m-%d')
    df_table = pd.DataFrame()
    if type(tickers)==str: tickers = [tickers]
    #tickers.append('BOLSASTGO.SN')
    for accion in tickers:
        df_ticker = yf.Ticker(accion)
        # get company name
        try:
            longname = df_ticker.info['longName']
            industry = df_ticker.info['industry']
            payoutRatio = df_ticker.info['payoutRatio']
        except:
            longname = accion
            industry = 'N/A'
            payoutRatio = 0 
        # get historical market data
        df = df_ticker.history(start=start, end=end, interval='1d')
        df['Ticker'] = accion
        df['Name'] = longname
        df['Industry'] = industry
        df['PayoutRatio'] = payoutRatio
        df['Pct Change - Close'] = df.Close.pct_change()
        df['Cummulative Returns'] = ((1 + df['Pct Change - Close']).cumprod() - 1)
        df_table = df_table.append(df)
    
    df_table.reset_index(inplace=True)
    df_table.sort_values(["Name", "Date"], axis=0, ascending=False, inplace=True) 
    df_table['Fecha'] = df_table['Date'].apply(lambda x:datetime.strftime(x,'%d/%m/%Y'))
    df_table = df_table[['Fecha','Name','Industry', 'Open', 'High', 'Low', 'Close', 'Volume', 'Pct Change - Close','Cummulative Returns']]
    table = go.Table(header=dict(values=list(df_table.columns),
                                 font=dict(size=10),
                                 #fill_color='paleturquoise',
                                 align='left'),
                     cells=dict(values=[df_table[k].tolist() for k in df_table.columns],
                                align = "left",
                                font=dict(size=10),
                               ),
                    )
    layout = go.Layout(title = 'Detalle de Información - '+'<b>'+col+'<b>', title_x=0.5,)
    fig = go.Figure(data=table, layout=layout)
    
    return fig
              
if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jan/2022 13:13:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 13:13:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 13:13:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 13:13:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 13:13:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 13:14:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 13:14:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 13:15:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 13:15:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 13:16:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 13:16:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2022 13:16:52] "POST /_dash-update-component

In [5]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import plotly.graph_objs as go
import pandas as pd
from datetime import datetime

import yfinance as yf

In [6]:
df_ticker = yf.Ticker('LTM.SN')

In [7]:
df_ticker.info

{'sector': 'Industrials',
 'fullTimeEmployees': 28701,
 'longBusinessSummary': 'LATAM Airlines Group S.A., together with its subsidiaries, provides passenger and cargo air transportation services in Peru, Argentina, the United States, Europe, Colombia, Brazil, Ecuador, Chile, the Asia Pacific, and rest of Latin America. The company provides passenger transport services to approximately 111 destinations in 16 countries, as well as cargo services to approximately 117 destinations in 20 countries; and operates loyalty programs. As of December 31, 2020, it operated a fleet of 297 aircraft, which include 286 passenger aircraft and 11 cargo aircraft; and subleased 3 aircraft comprising 2 passenger aircraft and 1 cargo aircraft to third parties. The company was formerly known as LAN Airlines S.A. and changed its name to LATAM Airlines Group S.A. in June 2012. LATAM Airlines Group S.A. was founded in 1929 and is headquartered in Santiago, Chile. On May 26, 2020, LATAM Airlines Group S.A., alon

In [8]:
df_ticker.institutional_holders

,Holder,Shares,Date Reported,% Out,Value
0,Vanguard International Stock Index-Emerging Ma...,3024840,2021-10-30,0.0050,3591392532
1,Vanguard International Stock Index-Total Intl ...,1964838,2021-10-30,0.0032,2332852157
2,Vanguard Intl Equity Index Fds-FTSE All World ...,932195,2021-10-30,0.0015,1106795123
3,DFA Investment Dimensions-DFA Emerging Mkts Value,736124,2021-10-30,0.0012,874000025
4,Cambria ETF Tr-Cambria Global Value ETF,355850,2021-11-29,0.0006,96079500
5,Russell Inv Co-Tax-Managed International Equit...,261943,2021-10-30,0.0004,311004923
6,Vanguard Intl Equity Index - Total World Stock...,130499,2021-10-30,0.0002,154941462
7,Vanguard World Fund-Vanguard ESG International...,46607,2021-08-30,0.0001,85323434


In [9]:
df_ticker.recommendations

In [10]:
df_ticker.actions

,Dividends,Stock Splits
Date,,
2010-08-16,0.217860,0.0
2011-01-10,182.497270,0.0
2011-05-16,0.030340,0.0
2011-09-12,78.363000,0.0
2012-01-09,126.909080,0.0
2012-05-14,26.370710,0.0
2013-05-13,3.217651,0.0
2017-05-15,22.996880,0.0
2018-05-10,0.076831,0.0


In [11]:
df_ticker.quarterly_financials

,2021-09-30,2021-06-30,2021-03-31,2020-12-31
Research Development,None,None,None,None
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,-785144000.0,-1127666000.0,-650600000.0,-1218706000.0
Minority Interest,-13872000.0,-10474000.0,-8658000.0,-6672000.0
Net Income,-691873000.0,-769637000.0,-430867000.0,-962477000.0
Selling General Administrative,172150000.0,154819000.0,148126000.0,255581000.0
Gross Profit,-81141000.0,-193763000.0,-201600000.0,-179555000.0
Ebit,-480256000.0,-313566000.0,-359380000.0,-569820000.0
Operating Income,-480256000.0,-313566000.0,-359380000.0,-569820000.0
Other Operating Expenses,226965000.0,-35016000.0,9654000.0,134684000.0


In [12]:
df_ticker.major_holders

,0,1
0,59.55%,% of Shares Held by All Insider
1,26.07%,% of Shares Held by Institutions
2,64.46%,% of Float Held by Institutions
3,22,Number of Institutions Holding Shares


In [13]:
df_ticker.balance_sheet

,2020-12-31,2019-12-31,2018-12-31,2017-12-31
Intangible Assets,1.046559e+09,1.448241e+09,1.441072e+09,1.617247e+09
Total Liab,1.809248e+10,1.795863e+10,1.663812e+10,1.453074e+10
Total Stockholder Equity,-2.435713e+09,3.130782e+09,3.360693e+09,4.176089e+09
Minority Interest,-6.672000e+06,-1.605000e+06,7.990800e+07,9.114700e+07
Other Current Liab,2.320468e+09,2.906855e+09,2.481659e+09,2.852657e+09
Total Assets,1.565009e+10,2.108781e+10,2.007872e+10,1.879797e+10
Common Stock,3.160718e+09,3.160718e+09,3.160718e+09,3.160718e+09
Other Current Assets,4.126530e+08,6.930570e+08,1.827610e+08,5.201580e+08
Retained Earnings,-4.193615e+09,3.522720e+08,2.189710e+08,4.751170e+08
Other Liab,2.400363e+09,2.467269e+09,2.346410e+09,2.082514e+09


In [14]:
df_ticker.calendar

,0,1
Earnings Date,2020-08-11 00:00:00,2020-08-17 00:00:00
Earnings Average,None,None
Earnings Low,None,None
Earnings High,None,None
Revenue Average,1802100000,1802100000
Revenue Low,1802100000,1802100000
Revenue High,1802100000,1802100000


In [15]:
df_ticker.sustainability

,Value
2020-3,
palmOil,False
controversialWeapons,False
gambling,False
socialScore,18.8
nuclear,False
furLeather,False
alcoholic,False
gmo,False
catholic,False
